In [57]:

import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [74]:
stocks = pd.read_csv('sp_500_stocks.csv')
print(stocks)
from sssecrets import IEX_CLOUD_API_TOKEN


    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
491    YUM
492    ZBH
493   ZBRA
494   ZION
495    ZTS

[496 rows x 1 columns]


In [75]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_f35e72cbeb9d4cc09281640e5f1f899a'
#f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token=sk_f35e72cbeb9d4cc09281640e5f1f899a'
data = requests.get(api_url).json()
data


{'companyName': 'Apple Inc',
 'marketcap': 3011012787200,
 'week52high': 199.62,
 'week52low': 123.15,
 'week52highSplitAdjustOnly': 199.62,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.480371316276438,
 'sharesOutstanding': 15552752000,
 'float': 0,
 'avg10Volume': 60146541,
 'avg30Volume': 54011435,
 'day200MovingAvg': 183.42,
 'day50MovingAvg': 190.8,
 'employees': 164000,
 'ttmEPS': 6.13,
 'ttmDividendRate': 0.9465536932059656,
 'dividendYield': 0.004889223621931641,
 'nextDividendDate': '',
 'exDividendDate': '2023-11-10',
 'nextEarningsDate': '2024-02-01',
 'peRatio': 31.04296909325223,
 'beta': 1.1395719249027279,
 'maxChangePercent': 74.67229518449031,
 'year5ChangePercent': 4.373851880431796,
 'year2ChangePercent': 0.115172042360409,
 'year1ChangePercent': 0.480371316276438,
 'ytdChangePercent': 0.5023614079361309,
 'month6ChangePercent': 0.04261247666777601,
 'month3ChangePercent': 0.11053493825517524,
 'month1ChangePercent': 0.011970100883383015,
 'day30ChangePerc

In [76]:
data['year1ChangePercent']

0.480371316276438

In [77]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
my_columns = ['Tickers', 'Price', 'One-Year Price Return','Number of Shares to Buy' ]


In [79]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token=sk_f35e72cbeb9d4cc09281640e5f1f899a'
    data = requests.get(batch_api_call_url).json()

    temp_data = []
    for symbol in symbol_string.split(','):
        temp_data.append([
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ])

    final_dataframe = pd.concat([final_dataframe, pd.DataFrame(temp_data, columns=my_columns)], ignore_index=True)

# Print the final DataFrame
final_dataframe

,Tickers,Price,One-Year Price Return,Number of Shares to Buy
0,A,139.57,-0.058125,N/A
1,AAL,14.31,0.125885,N/A
2,AAP,61.25,-0.563501,N/A
3,AAPL,193.60,0.480371,N/A
4,ABBV,154.94,-0.011654,N/A
...,...,...,...,...
491,YUM,130.00,0.027510,N/A
492,ZBH,120.56,-0.042910,N/A
493,ZBRA,269.41,0.085368,N/A
494,ZION,44.00,-0.046383,N/A


In [90]:
final_dataframe.sort_values('One-Year Price Return', ascending= False,inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)



ValueError: cannot insert level_0, already exists

In [113]:
def portfolio_input():
    global portfolio_size
    portfolio_size=input("Enter your portfolio value ")
    try:
        portfolio_size=float(portfolio_val)
    except ValueError:
        print('You need to enter real value ')
        portfolio_size=input("Enter your portfolio value ")
        portfolio_size=float(portfolio_val)
portfolio_input()


Enter your portfolio value 1000000000


In [124]:
print(portfolio_size)
position_size = portfolio_size*1000/len(final_dataframe.Price)
for i in range(0,len(final_dataframe.Price)):
    final_dataframe.loc[i,'Numberof Shares to Buy']=math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

10000.0


,Tickers,Price,One-Year Price Return,Number of Shares to Buy,Numberof Shares to Buy
0,LB,79.92,2.281287,N/A,2502.0
1,NVDA,488.30,2.212656,N/A,409.0
2,RCL,128.21,1.575015,N/A,1559.0
3,CCL,18.93,1.423816,N/A,10565.0
4,PHM,102.43,1.257221,N/A,1952.0
5,AMD,139.60,1.163670,N/A,1432.0
6,AVGO,1121.98,1.097880,N/A,178.0
7,CRM,266.34,1.057633,N/A,750.0
8,ANET,238.26,0.989313,N/A,839.0
9,GE,126.69,0.972870,N/A,1578.0


In [244]:
hqm_columns = ['Ticker', 'Price', 'Number of Shares to Buy',
               'One-Year Price Returns', 'One-Year Percentile',
               '6-months Price Returns', '6-months Percentile',
               '3-months Price Returns', '3-months Percentile',
               '1-month Price Returns', '1-month Percentile', 'HQM Score']

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token=sk_f35e72cbeb9d4cc09281640e5f1f899a'
    data = requests.get(batch_api_call_url).json()
    temp2_data = []

    for symbol in symbol_string.split(','):
        temp2_data.append([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ])

    hqm_dataframe = pd.concat([hqm_dataframe, pd.DataFrame(temp2_data, columns=hqm_columns)], ignore_index=True)

# Display the resulting DataFrame
hqm_dataframe



,Ticker,Price,Number of Shares to Buy,One-Year Price Returns,One-Year Percentile,6-months Price Returns,6-months Percentile,3-months Price Returns,3-months Percentile,1-month Price Returns,1-month Percentile,HQM Score
0,A,139.57,N/A,-0.058125,N/A,0.173325,N/A,0.255286,N/A,0.102275,N/A,N/A
1,AAL,14.31,N/A,0.125885,N/A,-0.119926,N/A,0.109302,N/A,0.162470,N/A,N/A
2,AAP,61.25,N/A,-0.563501,N/A,-0.059411,N/A,0.068579,N/A,0.153919,N/A,N/A
3,AAPL,193.60,N/A,0.480371,N/A,0.042612,N/A,0.110535,N/A,0.019108,N/A,N/A
4,ABBV,154.94,N/A,-0.011654,N/A,0.164109,N/A,0.024557,N/A,0.117329,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,130.00,N/A,0.027510,N/A,-0.032251,N/A,0.044919,N/A,0.016379,N/A,N/A
492,ZBH,120.56,N/A,-0.042910,N/A,-0.160901,N/A,0.033991,N/A,0.062858,N/A,N/A
493,ZBRA,269.41,N/A,0.085368,N/A,0.019218,N/A,0.186044,N/A,0.176874,N/A,N/A
494,ZION,44.00,N/A,-0.046383,N/A,0.693702,N/A,0.306026,N/A,0.280932,N/A,N/A


In [211]:
back_up = hqm_dataframe

In [254]:
import pandas as pd
from scipy import stats

time_periods = ['One-Year', '6-months', '3-months', '1-month']

for time_period in time_periods:
    returns_column = f'{time_period} Price Returns'
    percentile_column = f'{time_period} Percentile'

    # Exclude NaN values before calculating percentiles
    non_nan_returns = hqm_dataframe[returns_column].dropna()

    # Calculate percentile only if there are non-NaN values
    if not non_nan_returns.empty:
        hqm_dataframe[percentile_column] = hqm_dataframe[returns_column].apply(
            lambda x: stats.percentileofscore(non_nan_returns, x) / 100
        )
    else:
        # If all values are NaN, set percentiles to NaN
        hqm_dataframe[percentile_column] = 'N/A'
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Returns,One-Year Percentile,6-months Price Returns,6-months Percentile,3-months Price Returns,3-months Percentile,1-month Price Returns,1-month Percentile,HQM Score
0,A,139.57,N/A,-0.058125,0.211382,0.173325,0.693089,0.255286,0.867886,0.102275,0.693089,N/A
1,AAL,14.31,N/A,0.125885,0.542683,-0.119926,0.085366,0.109302,0.542683,0.162470,0.857724,N/A
2,AAP,61.25,N/A,-0.563501,0.006098,-0.059411,0.160569,0.068579,0.412602,0.153919,0.841463,N/A
3,AAPL,193.60,N/A,0.480371,0.880081,0.042612,0.396341,0.110535,0.548780,0.019108,0.284553,N/A
4,ABBV,154.94,N/A,-0.011654,0.313008,0.164109,0.670732,0.024557,0.270325,0.117329,0.739837,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,130.00,N/A,0.027510,0.406504,-0.032251,0.207317,0.044919,0.357724,0.016379,0.274390,N/A
492,ZBH,120.56,N/A,-0.042910,0.243902,-0.160901,0.056911,0.033991,0.315041,0.062858,0.483740,N/A
493,ZBRA,269.41,N/A,0.085368,0.481707,0.019218,0.339431,0.186044,0.739837,0.176874,0.894309,N/A
494,ZION,44.00,N/A,-0.046383,0.239837,0.693702,0.991870,0.306026,0.932927,0.280932,0.977642,N/A


In [269]:

from statistics import mean

momentum_percentile = []

for row in hqm_dataframe.index:
    temp_data3 = pd.DataFrame()
    for time_period in time_periods:
        temp_data3 = pd.concat([temp_data3, pd.DataFrame([hqm_dataframe.loc[row, f'{time_period} Percentile']])], axis=1)

    mean_percentile = temp_data3.mean(axis=1).iloc[0]
    hqm_dataframe.loc[row, 'HQM Score'] = mean_percentile

hqm_dataframe = hqm_dataframe.iloc[:, :12]
hqm_dataframe



,Ticker,Price,Number of Shares to Buy,One-Year Price Returns,One-Year Percentile,6-months Price Returns,6-months Percentile,3-months Price Returns,3-months Percentile,1-month Price Returns,1-month Percentile,HQM Score
0,A,139.57,N/A,-0.058125,0.211382,0.173325,0.693089,0.255286,0.867886,0.102275,0.693089,0.616362
1,AAL,14.31,N/A,0.125885,0.542683,-0.119926,0.085366,0.109302,0.542683,0.162470,0.857724,0.507114
2,AAP,61.25,N/A,-0.563501,0.006098,-0.059411,0.160569,0.068579,0.412602,0.153919,0.841463,0.355183
3,AAPL,193.60,N/A,0.480371,0.880081,0.042612,0.396341,0.110535,0.548780,0.019108,0.284553,0.527439
4,ABBV,154.94,N/A,-0.011654,0.313008,0.164109,0.670732,0.024557,0.270325,0.117329,0.739837,0.498476
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,130.00,N/A,0.027510,0.406504,-0.032251,0.207317,0.044919,0.357724,0.016379,0.274390,0.311484
492,ZBH,120.56,N/A,-0.042910,0.243902,-0.160901,0.056911,0.033991,0.315041,0.062858,0.483740,0.274898
493,ZBRA,269.41,N/A,0.085368,0.481707,0.019218,0.339431,0.186044,0.739837,0.176874,0.894309,0.613821
494,ZION,44.00,N/A,-0.046383,0.239837,0.693702,0.991870,0.306026,0.932927,0.280932,0.977642,0.785569


In [282]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)

In [274]:
portfolio_input()


Enter your portfolio value 1000000


In [283]:
position_size = float(portfolio_size)*100/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Returns,One-Year Percentile,6-months Price Returns,6-months Percentile,3-months Price Returns,3-months Percentile,1-month Price Returns,1-month Percentile,HQM Score
0,PVH,120.98,165,0.729959,0.959350,0.497655,0.981707,0.606039,0.997967,0.389725,0.995935,0.98374
1,RCL,128.21,155,1.575015,0.995935,0.309334,0.908537,0.405195,0.989837,0.227477,0.951220,0.961382
2,SLG,46.01,434,0.514109,0.896341,1.051051,0.997967,0.278098,0.904472,0.374307,0.993902,0.948171
3,URI,570.51,35,0.628789,0.930894,0.419923,0.963415,0.321152,0.947154,0.220838,0.945122,0.946646
4,PHM,102.43,195,1.257221,0.991870,0.348363,0.936992,0.388021,0.985772,0.164929,0.863821,0.944614
5,AVGO,1121.98,17,1.097880,0.987805,0.384766,0.951220,0.365753,0.975610,0.156761,0.849593,0.941057
6,EXPE,155.31,128,0.796946,0.967480,0.482107,0.977642,0.539705,0.995935,0.138469,0.817073,0.939533
7,VNO,29.17,685,0.397499,0.829268,1.028399,0.995935,0.314574,0.941057,0.352579,0.989837,0.939024
8,GPS,21.17,944,0.862310,0.975610,1.512760,1.000000,1.159851,1.000000,0.122481,0.747967,0.930894
9,STX,85.16,234,0.810759,0.971545,0.466668,0.971545,0.342479,0.959350,0.138270,0.815041,0.92937


In [302]:
hqm_dataframe.to_csv('Momentum_Strat.csv', index_label='Momentum')

writer = pd.ExcelWriter('Moment_Strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [295]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [298]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)
#for column in column_formats.keys():
 #   writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
  #  writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [299]:
writer.save()

AttributeError: 'XlsxWriter' object has no attribute 'save'